In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import random

In [5]:
# ------------------------WEB SCRAPING FROM FLIPKART (only function definition is here)-------------------------------------
# ----------------------------------------(Don't run unless urgent)---------------------------------------------------------
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    r = requests.get('https://www.flipkart.com/search?q=tshirts+for+men&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
#     print(soup)

    alls = []
    c=0
    for d in soup.findAll('div', attrs={'style':'width:25%'}):
#         ----------------------#to test code, restricted iterations-----------------------
#         c+=1
#         if c==10:
#             break
#         time.sleep(random.random()/100)
    
# -------------FINDING ALL DATA BY HTML PARSING AND CLASS IS FOUND MANUALLY FROM HTML CODE BY PRINTING SOUP----------------
        brand=d.find('div',attrs={'class':'_2B_pmu'})
#         print(brand)
        name=d.find('a',attrs={'class':'_2mylT6'})
        if name is "":
            name=d.find('a',attrs={'class':'_2mylT6 _3Ockxk'})
        price = d.find('div', attrs={'class':'_1vC4OE'})
#         print(price)

# ------------------------CREATING COLUMNS OF BRAND, NAME, PRICE, ETC BY GETTING TEXT FROM HTML TAGS------------------------
        all1=[]
        if brand is not None:
            all1.append(brand.text)
        else:
            all1.append("")
        
        if name is not None:
            all1.append(name.text)
        else:
            all1.append("")
    
        url = d.find('div', attrs={'class':'IIdQZO _1SSAGr'})
        if url==None:
#             all1.append("")
            continue
        n = url.find_all('a', attrs={'class':'_3dqZjq'},href=True)
        
# ------------------------------------VISITING URL OF EVERY PRODUCT AND GETTING FEATURES------------------------------------       
            
        r = requests.get('https://www.flipkart.com'+str(n[0]['href']), headers=headers)#, proxies=proxies)
        content2 = r.content
        soup2 = BeautifulSoup(content2)
        rt=''
        for rating in soup2.findAll('div',attrs={'class':'hGSR34 bqXGTW'}):
            rt=float(rating.text)
#             if rating is not None:
#                 all1.append(rating.text)
#             else:
#                 all1.append('')
        all1.append(rt)
        if price is not None:
            all1.append(float(price.text.replace('₹','').replace(',','')))
        else:
            all1.append('')
        
        all1.append('https://www.flipkart.com'+str(n[0]['href']))
        
        ratings=''
        for rt in soup2.findAll('span',attrs={'class':'_38sUEc _1je6zX'}):
            ratings=int(rt.text.split()[0].replace(',',''))
        all1.append(ratings)
        for box in soup2.findAll('div',attrs={'class':'_2GNeiG _2t27J6'}):
            for d3 in box.findAll('div',attrs={'class':'row'}):
                for d4 in d3.findAll('div',attrs={'class':'col col-3-12 _1kyh2f'}):
                    if d4.text=='Brand Color':
                        color=d3.find('div',attrs={'class':'col col-9-12 _1BMpvA'}).text
                    if d4.text=='Sleeve':
                        sleeve=d3.find('div',attrs={'class':'col col-9-12 _1BMpvA'}).text
                    if d4.text=='Neck Type':
                        neck=d3.find('div',attrs={'class':'col col-9-12 _1BMpvA'}).text
                    if d4.text=='Fabric':
                        fabric=d3.find('div',attrs={'class':'col col-9-12 _1BMpvA'}).text
                    if d4.text=='Brand Fit':
                        fit=d3.find('div',attrs={'class':'col col-9-12 _1BMpvA'}).text
                        
        if color is not None:
            all1.append(color)
        else:
            all1.append('')
            
        if sleeve is not None:
            all1.append(sleeve)    
        else:
            all1.append('')
            
        if neck is not None:
            all1.append(neck)
        else:
            all1.append('')
            
        if fabric is not None:
            all1.append(fabric)
        else:
            all1.append('')
            
        if fit is not None:
            all1.append(fit)
        else:
            all1.append('')
            
        all1.append(pageNo)
        alls.append(all1)
#         print(len(all1))
    return alls
get_data(1)

[]

In [6]:
# ------------------------------------------WEB SCRAPING FROM FLIPKART (FUNCTION CALL)--------------------------------------
# -----------------------------------------------(Don't run unless urgent)--------------------------------------------------
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Brand Name','Shirt Name','Rating','Price', 'URL','No. of ratings','Colour','Sleeve','Neck','Material','Fit Type','PG'])
df.to_csv('flipkart_products.csv', index=False, encoding='utf-8')
# df = pd.read_csv(r"C:\Users\Purva\Desktop\Jupyter Notebook\fktshirtformen.csv")
df

,Brand Name,Shirt Name,Rating,Price,URL,No. of ratings,Colour,Sleeve,Neck,Material,Fit Type,PG


In [7]:
# ------------------------WEB SCRAPING FROM AMAZON (only function definition is here)-------------------------------------
# ----------------------------------------(Don't run unless urgent)---------------------------------------------------------
no_pages = 1

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    r = requests.get('https://www.amazon.in/s?k=t-shirt&page='+str(pageNo)+'&qid=1594056503&ref=sr_pg_'+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)
    alls = []
    c=0
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-medium a-text-center'}):
        c+=1
#         if c==20:
#             break  #to test code, restricted iterations
        
        all1=[]
        #brand name
        brand=d.find('span',attrs={'class':'a-size-base-plus a-color-base','dir':'auto'})    
        if brand is not None:
            all1.append(brand.text)
        else:
            all1.append("")

        #shirt name    
        name=d.find('span',attrs={'class':'a-size-base-plus a-color-base a-text-normal', 'dir':'auto'})
        if name is not None:
            all1.append(name.text)
        else:
            all1.append("")
        
        #rating
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        if rating is not None:
            all1.append(rating.text)
        else:
            all1.append('')   
        
        #price
        price = d.find('span', attrs={'class':'a-offscreen'})
        if price is not None:
            all1.append(float(price.text.replace('₹','').replace(',','')))
        else:
            all1.append('')
            
        #visiting each product
        name = d.find('span', attrs={'class':'rush-component'})
        n = name.find_all('a', href=True)
        if n==[]:
            continue
        r = requests.get('https://www.amazon.in'+str(n[0]['href']), headers=headers)#, proxies=proxies)
        content2 = r.content
        soup2 = BeautifulSoup(content2)
        
        #link
        if n!=[]:
            all1.append('https://www.amazon.in'+str(n[0]['href']))  # add site link in table
        else:
            all1.append('')
        
        #no. of ratings
        ratings=soup2.find('span',attrs={'id':'acrCustomerReviewText','class':'a-size-base'}) 
        if ratings is not None:
            all1.append((ratings.text.replace(',','')))
        else:
            all1.append(0)
        
        
        #features and colour        
        s=""
        for d2 in soup2.findAll('div',attrs={'id':'feature-bullets', 'class':'a-section a-spacing-medium a-spacing-top-small'}):
            s=""
            for d3 in d2.findAll('span',attrs={'class':'a-list-item'}):
                s+=(str(d3).replace('</span>','').replace('<span class="a-list-item">','').strip())+"\n"
            all1.append(s.splitlines())

        
        colour=soup2.find('span',attrs={'class':'selection'})
        if colour is not None:
            all1.append(colour.text.replace('\n',''))
        else:
            all1.append("")
            

#         add all columns to table
        alls.append(all1)
    return alls

In [8]:

# ------------------------------------------WEB SCRAPING FROM AMAZON (FUNCTION CALL)----------------------------------------
# -----------------------------------------------(Don't run unless urgent)--------------------------------------------------
# import time
# start_time = time.time()
results = []
for i in range(4, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Brand Name','Shirt Name','Rating','Price','link','No. of ratings','Features','Colour'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')
# df = pd.read_csv(r"C:\Users\Purva\Desktop\Jupyter Notebook\amazonmix.csv")
# print(" time taken - %s seconds"%(time.time() - start_time))